**여기서부터는**

실제 동영상에서 뽑아온 R값, HSSC값을 통해 학습하는 것을 해본다.




우선 데이터를 불러와 30개씩 짤라서 출력해보자..

X_train 만드는 과정이다. 여기서 작업해야 할 부분은,
1. R값, HSSC값을 colab에 마운트한다.
2. 맨 위의 파일 이름만 수정하면 된다.

In [87]:
# 각각 Ra값, HSSC값 텍스트 파일은 열어서 읽어온다.
with open("Rvalue (21).txt", "r") as f:
  rvalue = f.read();

with open("HSSCvalue (21).txt", "r") as f:
  hsscvalue = f.read();

# 이때, 값들은 모두 문자열이고 ','로 구분이 되어 있기 때문에 split을 진행.
rList = rvalue.split(',')
hsscList = hsscvalue.split(',')
newRList = []
newHsscList = []
print(len(rList))
print(len(hsscList))

for rl in rList:
  if rl[0] == '[':
    rl = rl[1:]
  if rl[len(rl)-1] == ']':
    rl = rl[:len(rl)-2]
  real = round(float(rl),3)
  newRList.append(real)

for hl in hsscList:
  if hl[0] == '[':
    hl = hl[1:]
  if hl[len(hl)-1] == ']':
    hl = hl[:len(hl)-2]
  if hl == '':
    hl = 0
  real = round(float(hl),3)
  newHsscList.append(real)

print(newRList)
print(newHsscList)

96
95
[0.249, 0.304, 0.31, 0.297, 0.297, 0.314, 0.277, 0.343, 0.295, 0.295, 0.367, 0.371, 0.341, 0.326, 0.336, 0.341, 0.385, 0.385, 0.366, 0.366, 0.371, 0.381, 0.372, 0.392, 0.416, 0.415, 0.422, 0.428, 0.43, 0.431, 0.429, 0.427, 0.439, 0.438, 0.442, 0.44, 0.442, 0.456, 0.454, 0.452, 0.454, 0.456, 0.456, 0.45, 0.457, 0.465, 0.441, 0.443, 0.433, 0.427, 0.427, 0.426, 0.42, 0.409, 0.418, 0.41, 0.391, 0.381, 0.371, 0.371, 0.383, 0.381, 0.377, 0.376, 0.376, 0.373, 0.37, 0.367, 0.348, 0.348, 0.357, 0.339, 0.33, 0.334, 0.334, 0.357, 0.357, 0.357, 0.354, 0.354, 0.338, 0.319, 0.32, 0.313, 0.315, 0.314, 0.315, 0.315, 0.315, 0.315, 0.315, 0.315, 0.315, 0.315, 0.315, 0.315]
[-0.229, -0.041, -0.257, 0.0, -0.088, -0.43, -0.161, -0.16, 0.0, -0.269, -0.171, -0.194, -0.156, -0.139, -0.119, -0.217, 0.0, -0.308, 0.0, -0.094, -0.083, -0.09, -0.151, -0.183, -0.136, -0.053, -0.102, -0.065, -0.056, -0.044, 0.054, -0.05, -0.022, -0.062, 0.07, -0.041, -0.066, -0.049, 0.027, 0.003, -0.05, 0.0, 0.172, -0.033, 0.0

이제 y_train을 만드는 과정

맨 위 코드는 사용하지 말고, 그 다음 셀 사용. 선택지 3가지 있음.

맨 위는 낙상 + 정상이 합쳐졌을 때 사용하는 것으로 낙상이 계속 되다가 정상으로 되거나 그 반대의 경우 한쪽은 np.ones(a frame), 다른 한쪽은 np.zeros(b frame)으로 한 뒤 연결시켜줌.

두 번째는 그냥 낙상만 쭉 or 정상만 쭉.

세 번쨰는 뒤죽박죽 섞여 있어서 직접 입력

In [88]:
import numpy as np


# y_train data 입력(각 np.zeros(), np.ones()에 프레임 수 만큼 입력.모든 수의 합은 len(HsscList))
pre_fall = np.zeros(95)
fall = np.ones(0)
post_fall = np.zeros(0)
new_data = np.concatenate([pre_fall, fall, post_fall], axis=0)
for i in range(len(newHsscList)-30):
  y = new_data[i:i+30]
  y_train = np.load('y_train.npy')
  y_train = np.concatenate([y_train, y.reshape(1, -1)], axis=0)
  np.save('y_train.npy', y_train)


In [89]:
import numpy as np

for i in range(len(newHsscList)-30):
  x1 = np.array(newRList[i:i+30])
  x2 = np.array(newHsscList[i:i+30])
  re_x1 = np.expand_dims(x1, axis=1)
  re_x2 = np.expand_dims(x2, axis=1)
  merged = np.concatenate((re_x1, re_x2), axis=1)
  x = np.expand_dims(merged, axis=0)
  X_train = np.load('X_train.npy')
  X_train = np.concatenate((X_train, x), axis = 0)
  np.save('X_train.npy', X_train)

진행상황 shape 확인.

매번 X_train, y_train을 만들 때마다 아래 셀 실행시켜 확인.

In [91]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
print(X_train.shape)
print(y_train.shape)

(1928, 30, 2)
(1928, 30)


이제 밑은 학습시키는 과정.

In [92]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, GRU, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# 시계열 데이터 생성
time_steps = 30

X_train = np.load('X_train.npy')

y_train = np.load('y_train.npy')

history = []

model = Sequential()

model.add(GRU(32, return_sequences=True, input_shape=(30, 2)))
model.add(Dropout(0.2))
model.add(GRU(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])



model.fit(X_train, y_train, epochs=100, batch_size = 32)
print(X_train.shape, y_train.shape)
model.summary()

# test_loss, test_acc = model.evaluate(X_test, y_test)
# print('Test accuracy:', test_acc)

Epoch 1/100
61/61 [==============================] - 5s 9ms/step - loss: 0.3654 - accuracy: 0.8891
Epoch 2/100
61/61 [==============================] - 1s 9ms/step - loss: 0.2013 - accuracy: 0.9219
Epoch 3/100
61/61 [==============================] - 1s 9ms/step - loss: 0.1894 - accuracy: 0.9251
Epoch 4/100
61/61 [==============================] - 1s 9ms/step - loss: 0.1790 - accuracy: 0.9285
Epoch 5/100
61/61 [==============================] - 1s 9ms/step - loss: 0.1720 - accuracy: 0.9307
Epoch 6/100
61/61 [==============================] - 1s 12ms/step - loss: 0.1657 - accuracy: 0.9327
Epoch 7/100
61/61 [==============================] - 1s 13ms/step - loss: 0.1614 - accuracy: 0.9349
Epoch 8/100
61/61 [==============================] - 1s 13ms/step - loss: 0.1537 - accuracy: 0.9366
Epoch 9/100
61/61 [==============================] - 1s 13ms/step - loss: 0.1448 - accuracy: 0.9389
Epoch 10/100
61/61 [==============================] - 1s 9ms/step - loss: 0.1357 - accuracy: 0.9436
Epoch

In [ ]:
'''
사용한 데이터셋 종류
test5.mp4 -> 3개
test6.mp4 -> 2개
test8.mp4 -> 4개
test9.mov -> 2개
test10.mov -> 3개
test11.mov -> (낙상 43frame, 해제 89frame)

URFD-FALL
FALL1~5까지 완료. -> 여기까지 total 588개
FALL ~10까지 완료 -> 여기까지 total 1100개

추가 기록 내용 : 
FALL-13,14,16,18~(가방)은 MULTIPLEPOSES로 해야함. -> 
-> 결국 MULTIPLEPOSES로 해야하는거 제외하면 여기까지 total 1386개.


'''